In [1]:
# -*- coding: utf-8 -*-
"""
Created on Feb. 20, 2018
@author: Masker_Li
"""

import os
import sys
import numpy as np
from scipy import constants as Const
import pandas as pd


In [2]:
def Rename(fname,extraname):
    path=os.getcwd();
    OldName =os.path.basename(fname);
    if OldName[-8:] == '1sp-.log':
        NewName = OldName[:-8] + '1sp-_%s.log'%extraname;#新的文件名
        os.rename(OldName,NewName);  #重命名
    if OldName[-7:] == '1sp.log':
        NewName = OldName[:-7] + '1sp_%s.log'%extraname;#新的文件名
        os.rename(OldName,NewName);  #重命名
    if OldName[-8:] == '1sp+.log':
        NewName = OldName[:-8] + '1sp+_%s.log'%extraname;#新的文件名
        os.rename(OldName,NewName);  #重命名
    
        
def ReadLog(fn):
    Normals = False
    NAtoms = 1
    HLstart = -2
    HOMO = 0
    LUMO = 0
    NMO = 0
    FileNameStart = -3
    BondOrderStart = -10
    BondOrderList = [[]]
    Atom_labels_list = []
    QStart = -10
    QA = [0] # A clearly impossible value
    Origin_descriptors = []
##    is_stable=False     
    
    path=os.getcwd()
    fn = os.path.basename(fn)
    ifile = open( fn, 'r' )
    textlines = ifile.readlines()
    ifile.close()
    for line in textlines :
        words = line.split()
        if len(words) == 2: 
            if words[0] == '(Enter' and words[1][-13:] == 'g16/l101.exe)':
                FileNameStart = -2
        if FileNameStart > -3:
            FileNameStart += 1
            if FileNameStart == 1:
                Atom_labels_list += [int(x) for x in words[1:]]
                if len(Atom_labels_list) == 0:
                    Atom_labels_list = [1]
                FileNameStart = -3
                QA = [0] * len(Atom_labels_list)
                BondOrderList = [[]] * len(Atom_labels_list)
        
        if len(words) > 8: 
            if words[0] == 'NAtoms=':
                NAtoms = int(words[1])
        
        if len(words) > 3: 
            if words[0] == 'Normal' and words[1] == 'termination':
                Normals = True
            
        #HOMO,LUMO,QN
        if len(words) > 3 :
            if words[0] == 'Orbital' and words[1] == 'energies' and words[3] == 'kinetic' and words[5] =='(alpha):' :
                HLstart = 0
            if HLstart >= 0:
                if HLstart >= 1 and len(words) == 4:
                    if words[0].isdigit() == True and words[1][-1] == 'O':
                        HOMO = float(words[2])
                        #print(words[0],HLstart)
                        NMO = int(words[0])
                    if words[0] == str(NMO + 1) and words[1][-1] == 'V':
                        LUMO = float(words[2])
                        HLstart = -2
                HLstart += 1
        if len(words) == 5 :
            if words[2] == 'Natural' and words[3] == 'Population' and words[4] == 'Analysis:':
                QStart = -5
        if QStart >= -5 and QStart <= NAtoms and (0 in QA):
                            
            for i_atom in range(len(Atom_labels_list)):
                Atom_label = Atom_labels_list[i_atom]
                if len(words) > 5 and words[1] == str(Atom_label):
                    #print(QStart,words[1],words[2])
                    QA[i_atom] = float(words[2])
            QStart += 1
        
        if len(words) > 7 :
            if BondOrderList == [[]] * len(Atom_labels_list) and words[0] == 'Wiberg' and words[1] == 'bond' and words[2] == 'index' and words[3] == 'matrix':
                BondOrderStart = -1        
        if BondOrderStart >= -1 and BondOrderStart <= NAtoms + 3:
                            
            row_max = NAtoms //9 + 1
            #print(BondOrderStart,line)
            if BondOrderStart > 2:
                
                for i_atom in range(len(Atom_labels_list)):
                    Atom_label = Atom_labels_list[i_atom]
                    row_index = Atom_label // 9 
                    column_index = (Atom_label-1) % 9 + 1
                    if len(words) > 0 and BondOrderStart > ((NAtoms+3)*row_index + 2) and BondOrderStart < ((NAtoms+3)*(row_index+1)):
                        #print(BondOrderStart,line)
                        if float(words[column_index+1]) > 0.5:
#                        if float(words[column_index+1]) > 0.3:
                            #print(BondOrderList[i_atom])
                            BondOrderList[i_atom] = BondOrderList[i_atom] + [words[1],float(words[column_index+1])]
#                            BondOrderList[i_atom].append([words[1],float(words[column_index+1])])
#                            print(BondOrderStart,i_atom,BondOrderList)    
            BondOrderStart += 1   
            
            if BondOrderStart > (NAtoms+3)*row_max:
                BondOrderStart = -4
        
    fn_ls = fn.split('-')
    for i_atom in range(len(Atom_labels_list)):
        #print(BondOrderList[i_atom])
        Atom_label = Atom_labels_list[i_atom]
        fn = '-'.join([fn_ls[0],fn_ls[1],str(Atom_label),fn_ls[2]])   ##may need to be modified according to file name
#        fn = '-'.join([fn_ls[0],fn_ls[1],fn_ls[2],fn_ls[3],fn_ls[4],fn_ls[5],fn_ls[6]])
#        fn = '-'.join([fn_ls[1],str(Atom_label),fn_ls[2],fn_ls[3]])
        BondOrderList[i_atom] = BondOrderList[i_atom] + [None]*(8-len(BondOrderList[i_atom]))
        Origin_descriptors += [[fn,NAtoms,HOMO,LUMO,QA[i_atom]] + BondOrderList[i_atom]]

    if Normals == False:
        Rename(fn,'False')
    return Origin_descriptors

In [3]:
def CatchDscrptr(path,FQtype='N'):
    os.chdir(path)
    Output0 = []
    if FQtype == 'N':
        filetype = '*sp.log'
    elif FQtype == '+':
        filetype = '*sp+.log'
    elif FQtype == '-':
        filetype = '*sp-.log'
    from glob import glob
    for eachfile in glob(filetype) :
        for Origin_descriptor in ReadLog(eachfile):
            Output0.append(Origin_descriptor)
            print(eachfile,Output0[-1])
    Output = np.array(Output0)
    return Output


In [4]:
import os
import numpy as np
import pandas as pd

rootpath = r' '  ##root of the files with coordinates
os.chdir(rootpath)
Ar2Fixed_dir_list = os.listdir(rootpath)

EB_columns = ['filename','NAtoms','HOMO','LUMO', 'QA','A1','B1', 'A2','B2','A3','B3','A4','B4']
FQ_columns = ['filename','NAtoms','HOMO','LUMO', 'QA','HOMO+','LUMO+', 'QA+','HOMO-','LUMO-', 'QA-']
EB_table = np.empty([0,13])
FQ_table = np.empty([0,11])

EB_dir = r'%s'%rootpath
EB_table = np.r_[EB_table,CatchDscrptr(EB_dir,FQtype='N')]
FQ_dir = r'%s'%rootpath
Output = np.c_[np.c_[CatchDscrptr(FQ_dir,'N')[:,:5],CatchDscrptr(FQ_dir,'+')[:,2:5],CatchDscrptr(FQ_dir,'-')[:,2:5]]]
FQ_table = np.r_[FQ_table,Output]

EB_table_all = pd.DataFrame(EB_table,columns=EB_columns)
FQ_table_all = pd.DataFrame(FQ_table,columns=FQ_columns)
os.chdir(rootpath)
EB_table_all.to_csv('EB_table.csv')
FQ_table_all.to_csv('FQ_table.csv')

EB_table_all

POST-S09-P1-c1-1sp.log ['POST-S09-1-P1', 19, -0.236075, 0.071588, -0.06894, 'C', 1.0776, 'C', 1.0619, 'H', 0.9398, None, None]
POST-S09-P2-c10-1sp.log ['POST-S09-1-P2', 19, -0.255325, 0.070283, -0.27863, 'C', 1.0734, 'H', 0.9574, 'H', 0.9576, None, None]
POST-S09-P3-c1-1sp.log ['POST-S09-1-P3', 19, -0.236155, 0.073426, -0.06455, 'C', 1.0641, 'H', 0.9385, 'C', 1.0638, None, None]
POST-S19-P1-c5-1sp.log ['POST-S19-1-P1', 15, -0.233469, 0.072648, 0.09659, 'C', 1.0544, 'H', 0.9393, 'O', 1.0285, None, None]
POST-S19-P2-c7-1sp.log ['POST-S19-1-P2', 15, -0.231118, 0.073938, -0.08739, 'H', 0.9538, 'H', 0.9519, 'O', 1.027, None, None]
POST-S20-P1-c12-1sp.log ['POST-S20-1-P1', 35, -0.226136, -0.020312, 0.12574, 'C', 1.0376, 'C', 1.065, 'C', 1.0598, None, None]
POST-S26-P1-c1-1sp.log ['POST-S26-1-P1', 19, -0.198138, 0.063612, -0.03229, 'C', 1.0493, 'H', 0.9366, 'N', 1.1229, None, None]
POST-S26-P2-c5-1sp.log ['POST-S26-1-P2', 19, -0.22748, 0.053265, 0.12766, 'C', 1.0491, 'H', 0.9434, 'O', 1.0061,

,filename,NAtoms,HOMO,LUMO,QA,A1,B1,A2,B2,A3,B3,A4,B4
0,POST-S09-1-P1,19,-0.236075,0.071588,-0.06894,C,1.0776,C,1.0619,H,0.9398,None,None
1,POST-S09-1-P2,19,-0.255325,0.070283,-0.27863,C,1.0734,H,0.9574,H,0.9576,None,None
2,POST-S09-1-P3,19,-0.236155,0.073426,-0.06455,C,1.0641,H,0.9385,C,1.0638,None,None
3,POST-S19-1-P1,15,-0.233469,0.072648,0.09659,C,1.0544,H,0.9393,O,1.0285,None,None
4,POST-S19-1-P2,15,-0.231118,0.073938,-0.08739,H,0.9538,H,0.9519,O,1.027,None,None
5,POST-S20-1-P1,35,-0.226136,-0.020312,0.12574,C,1.0376,C,1.065,C,1.0598,None,None
6,POST-S26-1-P1,19,-0.198138,0.063612,-0.03229,C,1.0493,H,0.9366,N,1.1229,None,None
7,POST-S26-1-P2,19,-0.22748,0.053265,0.12766,C,1.0491,H,0.9434,O,1.0061,None,None
8,PRE-S09-1-c2,20,-0.364592,0.067684,-0.3876,C,1.0328,C,1.021,H,0.9283,H,0.9283
9,PRE-S09-2-c2,20,-0.364592,0.067684,-0.595,C,1.0328,H,0.9433,H,0.9433,H,0.9437
